In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

C:\Users\mahssalem\Anaconda3\envs\myEnv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in tqdm(glob.glob("midi_songs/*.mid")):
        midi = converter.parse(file)

        notes_to_parse = None

        parts = instrument.partitionByInstrument(midi)

        if parts: # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in tqdm(notes_to_parse):
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [3]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in tqdm(range(0, len(notes) - sequence_length, 1)):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [4]:
notes = get_notes()

n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

 36%|█████████████████████████████▋                                                    | 33/91 [00:09<00:16,  3.48it/s]


  0%|                                                                                          | 0/486 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████| 57077/57077 [00:00<00:00, 78239.44it/s]


In [5]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [6]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

In [7]:
model = create_network(network_input, n_vocab)

In [8]:
train(model, network_input, network_output)

Epoch 1/200


20032/57077 [=========>....................] - ETA: 1:44:34 - loss: 5.88 - ETA: 1:07:31 - loss: 5.87 - ETA: 55:09 - loss: 5.8575 - ETA: 48:58 - loss: 5.88 - ETA: 45:12 - loss: 5.86 - ETA: 42:42 - loss: 5.85 - ETA: 40:55 - loss: 5.81 - ETA: 39:33 - loss: 5.76 - ETA: 38:28 - loss: 5.73 - ETA: 37:37 - loss: 5.67 - ETA: 36:54 - loss: 5.60 - ETA: 36:19 - loss: 5.55 - ETA: 35:48 - loss: 5.50 - ETA: 35:21 - loss: 5.47 - ETA: 34:59 - loss: 5.44 - ETA: 34:39 - loss: 5.41 - ETA: 34:21 - loss: 5.38 - ETA: 34:05 - loss: 5.36 - ETA: 33:50 - loss: 5.34 - ETA: 33:36 - loss: 5.33 - ETA: 33:23 - loss: 5.30 - ETA: 33:11 - loss: 5.28 - ETA: 33:00 - loss: 5.26 - ETA: 32:50 - loss: 5.25 - ETA: 32:41 - loss: 5.23 - ETA: 32:32 - loss: 5.22 - ETA: 32:23 - loss: 5.21 - ETA: 32:15 - loss: 5.20 - ETA: 32:08 - loss: 5.20 - ETA: 32:00 - loss: 5.19 - ETA: 31:54 - loss: 5.29 - ETA: 31:47 - loss: 5.28 - ETA: 31:41 - loss: 5.26 - ETA: 31:35 - loss: 5.25 - ETA: 31:29 - loss: 5.23 - ETA: 31:23 - loss: 5.22 - ETA: 31:18 

24320/57077 [===========>..................] - ETA: 20:12 - loss: 4.74 - ETA: 20:10 - loss: 4.74 - ETA: 20:09 - loss: 4.74 - ETA: 20:06 - loss: 4.74 - ETA: 20:04 - loss: 4.74 - ETA: 20:02 - loss: 4.74 - ETA: 20:00 - loss: 4.74 - ETA: 19:58 - loss: 4.74 - ETA: 19:56 - loss: 4.74 - ETA: 19:53 - loss: 4.74 - ETA: 19:51 - loss: 4.74 - ETA: 19:49 - loss: 4.74 - ETA: 19:47 - loss: 4.74 - ETA: 19:45 - loss: 4.74 - ETA: 19:43 - loss: 4.74 - ETA: 19:41 - loss: 4.74 - ETA: 19:38 - loss: 4.74 - ETA: 19:36 - loss: 4.74 - ETA: 19:34 - loss: 4.74 - ETA: 19:32 - loss: 4.74 - ETA: 19:30 - loss: 4.73 - ETA: 19:28 - loss: 4.73 - ETA: 19:25 - loss: 4.73 - ETA: 19:23 - loss: 4.73 - ETA: 19:21 - loss: 4.73 - ETA: 19:19 - loss: 4.73 - ETA: 19:17 - loss: 4.73 - ETA: 19:15 - loss: 4.73 - ETA: 19:13 - loss: 4.73 - ETA: 19:11 - loss: 4.73 - ETA: 19:08 - loss: 4.73 - ETA: 19:06 - loss: 4.73 - ETA: 19:04 - loss: 4.73 - ETA: 19:02 - loss: 4.73 - ETA: 19:00 - loss: 4.73 - ETA: 18:58 - loss: 4.73 - ETA: 18:55 - loss

KeyboardInterrupt: 